In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import sys

In [2]:
def binarize(img):
    new_img = np.zeros(img.shape, np.int)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            new_img[i, j] = 255 if img[i, j] >= 128 else 0
    return new_img

In [35]:
class YokoiNumber(object):
    def __init__(self, bin_image):
        self.bin_image = bin_image
        self.size = bin_image.shape
        self.yokoi_array = None
        
    def h(self, b, c, d, e):
        if b == c and (d != b or e != b): return 'q'
        elif b == c and (d == b and e == b): return 'r'
        else: return 's'
    
    def f(self, a1, a2, a3, a4):
        a_tuple = [a1, a2, a3, a4]
        if all([a == 'r' for a in a_tuple]): return 5
        else: return sum([int(a == 'q') for a in a_tuple])
    
    def iterate(self):
        self.yokoi_array = -np.ones(self.size, np.int)
        for r in range(self.size[0]):
            for c in range(self.size[1]):
                if self.bin_image[r, c] == 0:
                    continue
                
                # define neighborhood from x0 to x8
                x0 = self.bin_image[r, c]
                x1 = self.bin_image[r, c+1] if c != self.size[1]-1 else 0
                x2 = self.bin_image[r-1, c] if r != 0 else 0
                x3 = self.bin_image[r, c-1] if c != 0 else 0
                x4 = self.bin_image[r+1, c] if r != self.size[0]-1 else 0
                x5 = self.bin_image[r+1, c+1] if r != self.size[0]-1 and c != self.size[1]-1 else 0
                x6 = self.bin_image[r-1, c+1] if r != 0 and c != self.size[1]-1 else 0
                x7 = self.bin_image[r-1, c-1] if r != 0 and c != 0 else 0
                x8 = self.bin_image[r+1, c-1] if r != self.size[0]-1 and c != 0 else 0
                
                # calculate h value
                a1 = self.h(x0, x1, x6, x2)
                a2 = self.h(x0, x2, x7, x3)
                a3 = self.h(x0, x3, x8, x4)
                a4 = self.h(x0, x4, x5, x1)
                
                self.yokoi_array[r, c] = self.f(a1, a2, a3, a4)
    
    def pprint(self, file=sys.stdout):
        for r in range(self.size[0]):
            for c in range(self.size[1]):
                if self.yokoi_array[r, c] <= 0:
                    file.write(' ')
                else:
                    file.write(str(self.yokoi_array[r,c]))
            file.write('\n')

In [10]:
image = cv2.imread('../lena.bmp', 0)
downsampled_image = image[::8, ::8]

bin_image = binarize(downsampled_image)

In [36]:
yokoi = YokoiNumber(bin_image)
yokoi.iterate()

In [39]:
yokoi.pprint(file=open('yokoi.txt', 'w'))